# Piano Playalong Generation from MIDI

## Description

Condensed version of the main processes. 
Fill in Hyperparameters and run to train model.

### Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import json
from pathlib import Path
from tqdm import tqdm
import random
import torch
from torch import nn, optim
from torch.utils.data import Dataset, ConcatDataset, DataLoader, random_split
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import lightning as L
# necessary cuda version on your system
print(torch.version.cuda)

#!pip install partitura miditok transformers[torch]

from torch.nn import functional as F
from transformers import GPT2LMHeadModel, AutoConfig, Trainer, TrainingArguments
from transformers.optimization import AdamW 
from transformers import pipeline, GenerationConfig, EarlyStoppingCallback


import partitura as pt
from miditok import Structured, TokenizerConfig
from miditok.data_augmentation import augment_dataset
from miditok.pytorch_data import DatasetMIDI, DataCollator
from miditok.pytorch_data import split_files_for_training
#!pip install pretty_midi jupyterlab fluidsynth
#!sudo apt-get install fluidsynth # you need program as well as package - see pretty-midi github
#!pip install pyFluidSynth
import fluidsynth
import pretty_midi 
from IPython.display import Audio
#!pip install wandb

import os
import scipy.io.wavfile as wavfile
#!pip install soundfile
import soundfile as sf
import matplotlib.pyplot as plt
# seed random module
L.seed_everything(21)


/home/seb/miniforge3/envs/bac/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


11.8


Seed set to 21


21

In [2]:
import wandb 
wandb.login(key="d19ead255e549ddbcbf7a80aaa3a634c4959656d")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: s-sonderegger. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/seb/.netrc


True

In [3]:
# Tokenizer parameters
TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": {(0, 4): 16, (4, 12): 4}, #time resolution per beat of first bar, and of second to third bar (max pause is 3 bars) 
    "num_velocities": 32,
    "special_tokens": ["PAD", "BOS", "EOS", "MASK"],
    "use_chords": False,
    "use_rests": False,
    "use_tempos": False,
    "use_time_signatures": False,
    "use_programs": False,
    "num_tempos": 32,  # number of tempo bins
    "tempo_range": (40, 200),  # (min, max)
}
# setup tokenizer

tok_config = TokenizerConfig(**TOKENIZER_PARAMS)
tokenizer = Structured(tok_config)

# Tokenization parameters

USE_BPE = False
MIDI_PATH = list(Path("data").glob("**/*.mid"))
LOOKUP_PATH = Path("data", "lookup.json")

# Midi preparation parameters

MAX_MIDI_LENGTH = 250 # number of notes per chunk, with no overlap ~ 2* the size of tokens in resulting chunk, else at least MAX_MIDI_LENGTH tokens up to 2* as many
AUGMENT_DATA = False  # if to apply data augmentation
OVERLAP = 1 # how many bars to overlap per chunk
dataset_chunks_dir = Path("data", "midi_chunks")

# Dataloader parameters

MAX_SEQUENCE_LENGTH = 300 # max length of tokenized midi in final training data (truncated and padded)

# load config and setup untrained model of gpt2
model_config = AutoConfig.from_pretrained("gpt2",vocab_size = len(tokenizer),
                                    bos_token_id = tokenizer["BOS_None"], 
                                    eos_token_id = tokenizer["EOS_None"],
                                    n_layer=16, # linear increase in model size (double = double)
                                    n_head=16,  # no increase in model size but probably training time
                                    n_embd=512, # most impact on model size (exp.)
                                    #n_positions=512, # max context length ,default = 1024, few additional parameters (~1-2%)
                                    #n_ctx = 512, # same as n_positions
                                    )


# set training args

training_args = TrainingArguments(
    output_dir = "./model/gpt-2",
    evaluation_strategy = "steps",
    eval_steps=200,
    eval_accumulation_steps=16,
    save_strategy="steps",
    save_steps=200,
    save_only_model=True, # False to resume training
    #auto_find_batch_size=True,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=10,
    learning_rate=1e-4,
    weight_decay=0.1,
    #max_grad_norm=2.0,
    gradient_accumulation_steps=12,
    lr_scheduler_type="cosine",
    warmup_ratio=0.025,    
    fp16=True, #only on cuda
    logging_steps=50,
    save_total_limit = 1, # saves only most recent checkpoint
    load_best_model_at_end= True, # + best one 
    report_to="wandb",    
    )

# setup logging

RUN_NAME = "bs10-lr1e-4-overlap-1-emb_d-512-len-midi-300-tok16"





/tmp/ipykernel_68681/4157335705.py:18: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  config = TokenizerConfig(**TOKENIZER_PARAMS)


{'PAD_None': 0,
 'BOS_None': 1,
 'EOS_None': 2,
 'MASK_None': 3,
 'Pitch_21': 4,
 'Pitch_22': 5,
 'Pitch_23': 6,
 'Pitch_24': 7,
 'Pitch_25': 8,
 'Pitch_26': 9,
 'Pitch_27': 10,
 'Pitch_28': 11,
 'Pitch_29': 12,
 'Pitch_30': 13,
 'Pitch_31': 14,
 'Pitch_32': 15,
 'Pitch_33': 16,
 'Pitch_34': 17,
 'Pitch_35': 18,
 'Pitch_36': 19,
 'Pitch_37': 20,
 'Pitch_38': 21,
 'Pitch_39': 22,
 'Pitch_40': 23,
 'Pitch_41': 24,
 'Pitch_42': 25,
 'Pitch_43': 26,
 'Pitch_44': 27,
 'Pitch_45': 28,
 'Pitch_46': 29,
 'Pitch_47': 30,
 'Pitch_48': 31,
 'Pitch_49': 32,
 'Pitch_50': 33,
 'Pitch_51': 34,
 'Pitch_52': 35,
 'Pitch_53': 36,
 'Pitch_54': 37,
 'Pitch_55': 38,
 'Pitch_56': 39,
 'Pitch_57': 40,
 'Pitch_58': 41,
 'Pitch_59': 42,
 'Pitch_60': 43,
 'Pitch_61': 44,
 'Pitch_62': 45,
 'Pitch_63': 46,
 'Pitch_64': 47,
 'Pitch_65': 48,
 'Pitch_66': 49,
 'Pitch_67': 50,
 'Pitch_68': 51,
 'Pitch_69': 52,
 'Pitch_70': 53,
 'Pitch_71': 54,
 'Pitch_72': 55,
 'Pitch_73': 56,
 'Pitch_74': 57,
 'Pitch_75': 58,
 'Pitc

### Load data and tokenize
Here you can filter out Midi-files you want to exclude (e.g. Tuning Tracks - Use lookup.json file to inspect data)

In [4]:
# files to exclude:
idx_del = []
lookup = json.loads(LOOKUP_PATH.read_text())
midi_paths_cleaned = [] 
# files containing this string will be excluded
lookup_str = "tuning"
for key, title in lookup.items():
    if lookup_str in title.lower():
        idx_del.append(key)
    # manually specify keys to exclude 
    elif str(key) in ["0021","0361","0362"]:   # verbal instructions and whole cds that couldnt be converted to midi (too long)
        idx_del.append(key)
    else:
        #exclude empty files
        #if not len(tokenizer(Path("data","midis", str(key)+".mid"))[0])>10:
        #    idx_del.append(key)
        #else:
        midi_paths_cleaned.append(Path("data","midis", str(key)+".mid"))

print(f"Loaded {len(midi_paths_cleaned)} valid files, {len(idx_del)} invalid files excluded.")


Loaded 1260 valid files, 55 invalid files excluded.


In [6]:
tokenizer_path = Path("data", "tokenizer", "tokenizer.json")

# load tokenizer if it already exists
if os.path.exists(tokenizer_path):
    tokenizer = Structured(params=tokenizer_path)
else:    
    # Builds the vocabulary with BPE
    if USE_BPE:
        print(f"Learning BPE...")
        tokenizer.learn_bpe(vocab_size=300, files_paths=midi_paths_cleaned)
        print(f"Saving tokenizer with BPE to {tokenizer_path}")
        tokenizer.save_params(tokenizer_path)
        print("Finished.")
    # Saves tokenizer without BPE
    else:
        print(f"Saving tokenizer to {tokenizer_path}")
        tokenizer.save_params(tokenizer_path)
        print("Done.")
"Vocab size:", len(tokenizer)



/home/seb/miniforge3/envs/bac/lib/python3.11/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/home/seb/miniforge3/envs/bac/lib/python3.11/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)


### Split MIDIs into subsequences

In [11]:
if not os.path.exists(dataset_chunks_dir) or not os.listdir(dataset_chunks_dir):
    midi_paths_chunks = split_files_for_training(
        files_paths=[path.resolve() for path in midi_paths_cleaned], # function needs absolute paths
        tokenizer=tokenizer,
        save_dir=dataset_chunks_dir,
        max_seq_len=MAX_MIDI_LENGTH,
        #average_num_tokens_per_note=4, # if not specified it is calculated from first 200 midis
        num_overlap_bars=OVERLAP,
     )
    if AUGMENT_DATA:
        augment_dataset(
        midi_paths_chunks,
        pitch_offsets=[-12, 12],
        velocity_offsets=[-4, 4],
        duration_offsets=[-0.5, 0.5],
    )
        
else: 
    midi_paths_chunks = sorted([Path(p) for p in dataset_chunks_dir.iterdir() if p.is_file()])[1:] #first object is some hidden file
midi_paths_chunks[:10]

[PosixPath('data/midi_chunks/0001_0.mid'),
 PosixPath('data/midi_chunks/0001_1.mid'),
 PosixPath('data/midi_chunks/0001_10.mid'),
 PosixPath('data/midi_chunks/0001_11.mid'),
 PosixPath('data/midi_chunks/0001_12.mid'),
 PosixPath('data/midi_chunks/0001_13.mid'),
 PosixPath('data/midi_chunks/0001_14.mid'),
 PosixPath('data/midi_chunks/0001_15.mid'),
 PosixPath('data/midi_chunks/0001_16.mid'),
 PosixPath('data/midi_chunks/0001_17.mid'),
 PosixPath('data/midi_chunks/0001_18.mid'),
 PosixPath('data/midi_chunks/0001_19.mid'),
 PosixPath('data/midi_chunks/0001_2.mid'),
 PosixPath('data/midi_chunks/0001_20.mid'),
 PosixPath('data/midi_chunks/0001_21.mid'),
 PosixPath('data/midi_chunks/0001_22.mid'),
 PosixPath('data/midi_chunks/0001_23.mid'),
 PosixPath('data/midi_chunks/0001_24.mid'),
 PosixPath('data/midi_chunks/0001_25.mid'),
 PosixPath('data/midi_chunks/0001_26.mid'),
 PosixPath('data/midi_chunks/0001_27.mid'),
 PosixPath('data/midi_chunks/0001_28.mid'),
 PosixPath('data/midi_chunks/0001_2

In [14]:
print(f"Total number of files after splitting into chunks: ",len(os.listdir(dataset_chunks_dir)))
# create train and validation-set 
val_chunks_paths = midi_paths_chunks[0:len(midi_paths_chunks)//5]
train_chunks_paths = midi_paths_chunks[len(midi_paths_chunks)//5:]

print(f"Size after splitting into train/val : {len(train_chunks_paths)} / {len(val_chunks_paths)}")

Total number of files after splitting into chunks:  79041
Size after splitting into train/val : 63232 / 15808


### Dataloading and Collator

In [15]:
dataset_train = DatasetMIDI(
    files_paths=train_chunks_paths,
    max_seq_len=MAX_SEQUENCE_LENGTH,
    tokenizer=tokenizer,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"]
)

dataset_val = DatasetMIDI(
    files_paths=val_chunks_paths,
    max_seq_len=MAX_SEQUENCE_LENGTH,
    tokenizer=tokenizer,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"]
)
# setup Collator
collator = DataCollator(
    pad_token_id=tokenizer.pad_token_id, pad_on_left=False, copy_inputs_as_labels=True , shift_labels=False) #padding on left is not necessary for gpt2

# Set up dataloader
data_loader_train = DataLoader(dataset=dataset_train, batch_size=64, collate_fn=collator,shuffle=True) #set num workers for faster loading?
data_loader_val = DataLoader(dataset=dataset_val, batch_size=64, collate_fn=collator,shuffle=True)
print(f"Dataloader created.")
print(f"N samples in train/val : {len(data_loader_train)*64} / {len(data_loader_val)*64}")

Dataloader created.
N samples in train/val : 63232 / 15808


## Training the model

### Transformer model class

In [17]:
# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)


Device: cuda:0


In [18]:
model = GPT2LMHeadModel(model_config)
print("Number of Parameters in model:", f'{sum(p.numel() for p in model.parameters())/1e6:.2f}',"M",model.config)

Number of Parameters in model: 51.16 M GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 1,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 512,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 16,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.39.3",
  "use_cache": true,
  "vocab_size": 378
}



In [19]:
# init wandb run 
run = wandb.init(project="bac",name=RUN_NAME)

#log model 
log_data= {"model_type" : "gpt-2",
           "model_params": f'{sum(p.numel() for p in model.parameters())/1e6:.3f}M',
           "n_layer": model.config.n_layer,
           "n_head": model.config.n_head,
           "n_embd" : model.config.n_embd,
           "n_ctx" : model.config.n_ctx,
           "batch_size" : training_args.per_device_train_batch_size,
           "lr" : training_args.learning_rate,
           "lr-scheduler": training_args.lr_scheduler_type,
           "num_epochs": training_args.num_train_epochs,
           "weight_decay": training_args.weight_decay,
           "warmup_ratio" : training_args.warmup_ratio,
           "early_stopping_patience" : 1,
           "tokenizer_type": "Structured",
           "tokenizer_vocab_size" : len(tokenizer),  
           "overlap" : OVERLAP,           
           "pad_on_left" : False,
           "max_seq_length" : MAX_SEQUENCE_LENGTH,
           "augment_data" : False,    
           "max_midi_length" : MAX_MIDI_LENGTH,                
           }
table = wandb.Table(columns=["hp","value"])
for key, value in log_data.items():
    table.add_data(key,str(value))
log_name = f"Training Parameters-{run.name}"
wandb.log({log_name: table})

# setup trainer
trainer = Trainer(
    model=model, 
    tokenizer= tokenizer, 
    args = training_args,
    data_collator=collator,
    train_dataset=dataset_train,
    eval_dataset=dataset_val, 
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)],
)

# train 
trainer.train()

/home/seb/miniforge3/envs/bac/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  1%|          | 50/5270 [08:11<14:01:07,  9.67s/it]

{'loss': 4.2673, 'grad_norm': 1.190010905265808, 'learning_rate': 3.787878787878788e-05, 'epoch': 0.09}


  2%|▏         | 100/5270 [16:15<13:56:36,  9.71s/it]

{'loss': 3.0045, 'grad_norm': 2.0781729221343994, 'learning_rate': 7.575757575757576e-05, 'epoch': 0.19}


  3%|▎         | 150/5270 [24:19<13:44:24,  9.66s/it]

...

 76%|███████▌  | 4000/5270 [13:06:21<3:24:37,  9.67s/it]

{'loss': 1.8428, 'grad_norm': 0.6057270765304565, 'learning_rate': 1.4332596999203068e-05, 'epoch': 7.59}


                                                        
 76%|███████▌  | 4000/5270 [13:13:48<3:24:37,  9.67s/it]

{'eval_loss': 1.9187806844711304, 'eval_runtime': 446.7364, 'eval_samples_per_second': 35.386, 'eval_steps_per_second': 3.539, 'epoch': 7.59}


 77%|███████▋  | 4034/5270 [13:19:22<3:23:24,  9.87s/it]  

In [19]:
# finish logging
wandb.finish()

eval/loss,█▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▃███████
eval/samples_per_second,█████▇████▇█████████▆▁▁▁▁▁▁▁
eval/steps_per_second,█████▇████▇█████████▆▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,█▃▂▅▄▃▃▂▃▂▁▂▂▃▁▂▁▂▁▂▁▁▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▁▆████████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
train/loss,█▅▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,2.15159
eval/runtime,436.759


## Inference

In [20]:
model_local = GPT2LMHeadModel.from_pretrained("model/gpt-2/checkpoint-5400")
tokenizer_path = Path("data", "tokenizer", "tokenizer.json")

# load tokenizer if it already exists
if os.path.exists(tokenizer_path):
    tokenizer = Structured(params=tokenizer_path)
    
model_local.eval()
#what to give as input
num_notes = 150
gen_config = GenerationConfig(bos_token_id=1,
                            max_length=(4*num_notes)+1, # +1 for bos token
                            do_sample=True, # False = greedy generation
                            temperature=1.5, 
                            #top_k = 50,
                            top_p=0.9,  
                            #num_beam_groups=2, 
                            num_beams = 1, # 1 = beam search disabled                         
                            #diversity_penalty= 2.5, # for beam search, needs beam-groups
                            #early_stopping=True, 
                            #repetition_penalty=1.5,
                            #no_repeat_ngram_size=4, # ? no token is ever repeated
                            eos_token_id=2,
                            pad_token_id=0,
                            )
out_no_in = model_local.generate(generation_config=gen_config)
print(out_no_in.size(),out_no_in,[tokenizer[i]for i in out_no_in.numpy().flatten().tolist()])

/home/seb/miniforge3/envs/bac/lib/python3.11/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/home/seb/miniforge3/envs/bac/lib/python3.11/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)


torch.Size([1, 601]) tensor([[  1, 404,  59, 107, 128, 284,  51, 107, 127, 284,  42, 106, 127, 285,
          43, 105, 126, 289,  62, 110, 149, 304,  40, 107, 131, 284,  55, 107,
         132, 295,  43, 107, 135, 287,  47, 106, 131, 284,  54, 107, 134, 302,
          38, 104, 128, 306,  40, 107, 131, 284,  54, 107, 131, 306,  41, 106,
         136, 284,  36, 107, 131, 307,  43, 106, 144, 284,  55, 107, 134, 307,
          48, 101, 132, 284,  57, 101, 125, 306,  41, 105, 148, 286,  33, 106,
         145, 286,  55, 107, 141, 305,  26, 108, 124, 284,  47, 111, 132, 297,
          41, 106, 133, 299,  38, 105, 129, 291,  40, 107, 129, 294,  35, 103,
         138, 302,  50, 108, 130, 290,  50, 111, 136, 297,  46, 109, 137, 298,
          47, 110, 131, 295,  45, 104, 137, 284,  54, 106, 140, 286,  49, 104,
         137, 299,  52, 103, 127, 286,  57, 100, 139, 291,  52, 111, 144, 313,
          41, 103, 126, 286,  43, 101, 133, 310,  43,  96, 137, 294,  35, 105,
         128, 302,  40, 105, 13

### Inference with inputs

In [21]:
input_midi = Path("data","inputs","I-II-V_fmaj.mid") # arpeggio_cmaj.mid, I-II-V_fmaj.mid , single_chord_cmaj.mid
input_tokens = tokenizer.encode(input_midi,)
print((input_tokens))
input_tokens = torch.concat((torch.tensor([[1]]),torch.tensor(input_tokens)),dim=1) # add bos token

out_in = model_local.generate(inputs=input_tokens, generation_config=gen_config)


[TokSequence(tokens=['TimeShift_3.12.32', 'Pitch_57', 'Velocity_79', 'Duration_3.3.32', 'TimeShift_0.1.32', 'Pitch_53', 'Velocity_75', 'Duration_3.3.32', 'TimeShift_0.0.1', 'Pitch_64', 'Velocity_87', 'Duration_3.6.32', 'TimeShift_0.1.32', 'Pitch_41', 'Velocity_67', 'Duration_2.31.32', 'TimeShift_0.0.1', 'Pitch_45', 'Velocity_59', 'Duration_3.0.32', 'TimeShift_3.28.32', 'Pitch_65', 'Velocity_39', 'Duration_3.16.32', 'TimeShift_0.1.32', 'Pitch_43', 'Velocity_75', 'Duration_2.31.32', 'TimeShift_0.0.1', 'Pitch_58', 'Velocity_87', 'Duration_3.11.32', 'TimeShift_0.1.32', 'Pitch_55', 'Velocity_95', 'Duration_3.10.32', 'TimeShift_4.1.4', 'Pitch_58', 'Velocity_79', 'Duration_3.6.32', 'TimeShift_0.1.32', 'Pitch_55', 'Velocity_79', 'Duration_3.6.32', 'TimeShift_0.0.1', 'Pitch_64', 'Velocity_83', 'Duration_3.5.32', 'TimeShift_0.1.32', 'Pitch_48', 'Velocity_51', 'Duration_3.5.32'], ids=[392, 40, 111, 222, 285, 36, 110, 222, 284, 47, 113, 225, 285, 24, 108, 218, 284, 28, 106, 219, 408, 48, 101, 235,

In [22]:
print(out_in.size(),out_in,[tokenizer[i]for i in out_in.numpy().flatten().tolist()])

torch.Size([1, 601]) tensor([[  1, 392,  40, 111, 222, 285,  36, 110, 222, 284,  47, 113, 225, 285,
          24, 108, 218, 284,  28, 106, 219, 408,  48, 101, 235, 285,  26, 110,
         218, 284,  41, 113, 230, 285,  38, 115, 229, 413,  41, 111, 225, 285,
          38, 111, 225, 284,  47, 112, 224, 285,  31, 104, 224, 416,  59, 117,
         132, 284,  41, 112, 126, 284,  38, 109, 136, 284,  50, 112, 138, 290,
          43, 111, 136, 310,  43, 105, 126, 297,  62, 116, 124, 284,  41, 111,
         128, 294,  62, 111, 131, 294,  62, 115, 145, 296,  40, 112, 139, 303,
          35, 107, 126, 288,  51, 113, 126, 300,  50, 106, 124, 286,  54, 107,
         124, 289,  40, 109, 127, 296,  47, 110, 138, 308,  43, 105, 156, 318,
          55, 111, 129, 304,  40, 109, 126, 284,  52, 109, 127, 289,  59, 114,
         138, 299,  50, 112, 139, 300,  52, 108, 137, 297,  57, 106, 145, 284,
          52, 110, 156, 296,  45, 104, 127, 293,  53, 105, 124, 292,  52, 107,
         126, 284,  54, 107, 12

## Back to MIDI

In [24]:
#sequence = [[1, 226, 43, 110, 124, 188, 48, 110, 124, 188, 38, 110, 124, 188, 53, 111, 124, 188, 57, 110]]
#print(sequence.tolist())

out = tokenizer.tokens_to_midi(tokens=out_in)
print(out.note_num(),out)
fpath = os.path.join("data","example","example.mid")
out.dump_midi(fpath)
midi_data = pretty_midi.PrettyMIDI(midi_file=fpath, initial_tempo=100)
audio_data = midi_data.fluidsynth(fs=44100, sf2_path=os.path.join("data","soundfont","SC55_zzdenis_v0.5.sf2"))
print(len(audio_data),type(audio_data))
wav_path = f"{fpath[:-3]}wav"
wavfile.write(wav_path,44100,audio_data)
audio,sr = sf.read(wav_path)
Audio(audio,rate=sr)



/tmp/ipykernel_60636/766024925.py:4: UserWarning: miditok: The `tokens_to_midi` method had been renamed `decode`. It is now depreciated and will be removed in future updates.
  out = tokenizer.tokens_to_midi(tokens=out_in)


150 Score(ttype=Tick, tpq=32, begin=0, end=1806, tracks=1, notes=150, time_sig=1, key_sig=0, markers=0, lyrics=0)
1288547 <class 'numpy.ndarray'>
